<a href="https://colab.research.google.com/github/NikhilHooda/EPL_Web_Scrapper/blob/main/EPL_Web_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import random
import time

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 11; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.127 Mobile Safari/537.36",
    "Mozilla/5.0 (iPad; CPU OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15A5341f Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:88.0) Gecko/20100101 Firefox/88.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Linux; U; Android 4.4.4; en-us; Nexus 5 Build/KTU84P) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/33.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.48",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 7.1.2; SM-T590) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.111 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Mozilla/5.0 (X11; Linux i686; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.6 Mobile/15E148 Safari/604.1"
]

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Select a random user-agent for each request
headers = {"User-Agent": random.choice(user_agents)}

time.sleep(random.uniform(10, 30))

# Send the request with a random user agent
data = requests.get(standings_url, headers=headers)

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(data.text) ## parse html code

In [5]:
standings_table = soup.select('table.stats_table')[0] ## parse html code for first team in EPL stats table

In [6]:
links = standings_table.find_all('a') ## all team links are a tags

In [7]:
links = [l.get("href") for l in links] ## make links a list of href links

In [8]:
links = [l for l in links if '/squads/' in l] ## filter links for only squad links

In [9]:
team_urls = [f"https://fbref.com{l}" for l in links] ## add on the start part of link to make it a full link

In [10]:
team_url = team_urls[0] ## access first teams link

In [11]:
data = requests.get(team_url)

In [12]:
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202410310" />\n<script>\n/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */\n  window.dataLayer = window.dataLayer ||[];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'consent\',\'default\',{\n        \'ad_storage\':\'denied\',\n        \'analytics_storage\':\'denied\',\n        \'ad_user_data\':\'denied\',\n        \'ad_personalization\':\'denied\',\n        \'personalization_storage\':\'denied\',\n        \'functionality_storage\':\'granted\',\n        \'security_storage\':\'granted\',\n        \'wait_for_update\': 500\n      });\n      gtag("set", "ads_data_re

In [13]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

<ipython-input-13-006488a1d5e0>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")


In [14]:
matches[0]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,NaN,NaN,56.0,78146.0,Rúben Dias,4-3-3,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,0.8,1.0,52.0,39818.0,Kevin De Bruyne,3-2-4-1,4-2-3-1,Anthony Taylor,Match Report,NaN
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,3.3,0.3,75.0,53147.0,Kevin De Bruyne,3-2-4-1,5-4-1,Samuel Allison,Match Report,NaN
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,3.0,0.7,67.0,62469.0,Kevin De Bruyne,3-2-4-1,4-2-3-1,Michael Oliver,Match Report,NaN
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,2.1,1.0,54.0,52148.0,Kyle Walker,4-2-3-1,5-3-2,Darren Bond,Match Report,NaN
5,2024-09-18,20:00,Champions Lg,League phase,Wed,Home,D,0,0,it Inter,2.3,0.7,60.0,50922.0,Kevin De Bruyne,3-2-4-1,3-5-2,Glenn Nyberg,Match Report,NaN
6,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Home,D,2,2,Arsenal,2.1,0.7,77.0,52846.0,Kyle Walker,4-3-3,4-4-2,Michael Oliver,Match Report,NaN
7,2024-09-24,19:45,EFL Cup,Third round,Tue,Home,W,2,1,Watford,NaN,NaN,72.0,40584.0,Kyle Walker,3-2-4-1,5-3-2,David Webb,Match Report,NaN
8,2024-09-28,12:30,Premier League,Matchweek 6,Sat,Away,D,1,1,Newcastle Utd,0.9,1.6,62.0,52248.0,Kyle Walker,4-2-3-1,4-3-3,Jarred Gillett,Match Report,NaN
9,2024-10-01,21:00,Champions Lg,League phase,Tue,Away,W,4,0,sk Slovan Bratislava,3.7,0.1,75.0,22500.0,İlkay Gündoğan,4-2-3-1,4-2-3-1,Davide Massa,Match Report,NaN


In [15]:
soup = BeautifulSoup(data.text)

In [16]:
links = soup.find_all('a')

In [17]:
links = [l.get("href") for l in links]

In [18]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [19]:
links

['/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [20]:
data = requests.get(f"https://fbref.com{links[0]}")

In [21]:
shooting = pd.read_html(data.text, match="Shooting")[0]

<ipython-input-21-8da1f2d296b3>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [22]:
shooting.head()

For Manchester City                                                        \
                 Date   Time                 Comp                Round  Day   
0          2024-08-10  15:00  FA Community Shield  FA Community Shield  Sat   
1          2024-08-18  16:30       Premier League          Matchweek 1  Sun   
2          2024-08-24  15:00       Premier League          Matchweek 2  Sat   
3          2024-08-31  17:30       Premier League          Matchweek 3  Sat   
4          2024-09-14  15:00       Premier League          Matchweek 4  Sat   

                                              ... Standard                \
  Venue Result     GF     GA        Opponent  ...     Dist   FK PK PKatt   
0  Home      D  1 (7)  1 (6)  Manchester Utd  ...      NaN  NaN  0     0   
1  Away      W      2      0         Chelsea  ...     19.1  0.0  0     0   
2  Home      W      4      1    Ipswich Town  ...     17.8  1.0  1     1   
3  Away      W      3      1        West Ham  ...     15.0  1.0  0     0   
4  Home      W      2      1       Brentford  ...     17.2  0.0  0     0   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      0.8  0.8    0.07  1.2     1.2        Match Report  
2      3.3  2.6    0.20  0.7     0.4        Match Report  
3      3.0  3.0    0.13  0.0     0.0        Match Report  
4      2.1  2.1    0.12 -0.1    -0.1        Match Report  

[5 rows x 26 columns]

In [23]:
shooting.columns = shooting.columns.droplevel() ## to drop the top bolded header row bc we dont need

In [24]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date") ## merge team and shooting data together into one dataframe

In [25]:
matches[0].shape

(49, 20)

In [26]:
years = list(range(2024, 2014, -1))

In [27]:
all_matches = []

In [28]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [30]:
import time
time.sleep(random.uniform(10, 30))
headers = {"User-Agent": random.choice(user_agents)}
data = requests.get(standings_url, headers=headers)

for year in years:
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    headers = {"User-Agent": random.choice(user_agents)}
    time.sleep(random.uniform(10, 30))
    data = requests.get(team_url, headers=headers)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    headers = {"User-Agent": random.choice(user_agents)}
    time.sleep(random.uniform(10, 30))
    data = requests.get(f"https://fbref.com{links[0]}", headers=headers)
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
      continue

    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(15) ## prevent delays from too many requests too

KeyboardInterrupt: 

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Match Report,NaN,11,5,19.1,0.0,0,0,2024,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Match Report,NaN,13,4,17.8,1.0,1,1,2024,Manchester City
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,23,8,15.0,1.0,0,0,2024,Manchester City
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,Match Report,NaN,18,7,17.2,0.0,0,0,2024,Manchester City
6,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Home,D,2,2,Arsenal,...,Match Report,NaN,33,11,20.2,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Away,L,1.0,3.0,Aston Villa,...,Match Report,NaN,10,4,14.9,0.0,0,0,2024,Wolverhampton Wanderers
7,2024-09-28,17:30,Premier League,Matchweek 6,Sat,Home,L,1.0,2.0,Liverpool,...,Match Report,NaN,8,3,22.9,0.0,0,0,2024,Wolverhampton Wanderers
8,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Away,L,3.0,5.0,Brentford,...,Match Report,NaN,17,6,20.9,0.0,0,0,2024,Wolverhampton Wanderers
9,2024-10-20,14:00,Premier League,Matchweek 8,Sun,Home,L,1.0,2.0,Manchester City,...,Match Report,NaN,3,2,17.6,0.0,0,0,2024,Wolverhampton Wanderers


In [ ]:
match_df.to_csv("2014-24matches.csv")
from google.colab import files
files.download("2014-24matches.csv")